In [1]:
import numpy as np
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from ecit import *


def ecit_simu_alpha(n_list=[2000, 4000],
              t = 1000,
              cit = kcit,
              k = 10,
              alphas=[0.5, 0.75, 1, 1.25, 1.5, 1.75, 2],
              show_progress=True):
    
    results = []

    for n in n_list:

        eI = [0] * len(alphas)
        eII = [0] * len(alphas)
        tim = 0
        start_time = time.time()
        iteration_loop = tqdm(range(t), desc=f"n={n}", leave=True, dynamic_ncols=True) if show_progress else range(t)

        for sed in iteration_loop:
            #np.random.seed(sed)
            #random.seed(sed)
            dataI = np.hstack((generate_samples(n=n,indp='C',z_dis='gaussian',noise_dis='laplace',noise_std=0.3)))
            dataII = np.hstack((generate_samples(n=n,indp='N',z_dis='gaussian',noise_dis='laplace',noise_std=0.3)))

            obj_ECIT = ECIT(dataI, cit, p_stable, k)
            _, pI_list = obj_ECIT([0], [1], [2], return_p_list=True)
            obj_ECIT = ECIT(dataII, cit, p_stable, k)
            _, pII_list = obj_ECIT([0], [1], [2], return_p_list=True)

            for j, alpha in enumerate(alphas):
                ensfunc = lambda p_list: p_stable(p_list, alpha=alpha)
                pI = ensfunc(pI_list)
                pII = ensfunc(pII_list)
                if pI<0.05:
                    eI[j] += 1
                if pII>0.05:
                    eII[j] += 1
    

        end_time = time.time()
        tim = end_time - start_time
        eI = [count / t for count in eI]
        eII = [count / t for count in eII]

        results.append({
            'n': n,
            'eI': eI,
            'eII': eII,
            'tim': tim
        })

    return results




def show_results_choose(results, alphas):
    
    fig, axes = plt.subplots(1, 3, figsize=(16, 4), dpi=500, sharex=True)

    ax_eI, ax_eII, ax_eSum = axes

    colors = cm.viridis(np.linspace(0.25, 0.85, len(results)))

    for res_dir, color in zip(results, colors):
        
        label = f"N = {res_dir['n']}"

        eI = np.array(res_dir['eI'])
        eII = np.array(res_dir['eII'])
        eSum = eI + eII

        ax_eI.plot(alphas, eI, label=label, color=color)
        ax_eII.plot(alphas, eII, color=color)
        ax_eSum.plot(alphas, eSum, color=color)

    ax_eI.set_ylim(0, 1)
    ax_eII.set_ylim(0, 1)
    ax_eSum.set_ylim(0, 1)
    
    ax_eI.set_ylabel("Error Rate")
    ax_eI.set_title("Type I Error")
    #ax_eI.set_xlabel("Sample Size (n)")
    
    ax_eII.set_title("Type II Error")
    #ax_eII.set_xlabel("Sample Size (n)")

    ax_eSum.set_title("Total Error Rate")
    #ax_eSum.set_xlabel("Sample Size (n)")

    fig.supxlabel("alpha", fontsize=11)
    fig.legend(loc='upper center', bbox_to_anchor=(0.51, -0.05), ncol=2, fontsize=12)

    ax_eI.set_xticks(alphas)
    ax_eII.set_xticks(alphas)
    ax_eSum.set_xticks(alphas)

    plt.tight_layout()
    plt.show()

/Users/gzk/anaconda3/envs/graph/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [37]:
n_list=[2000, 4000]
alphas=[0.5, 0.75, 1, 1.25, 1.5, 1.75, 2]
t = 100
k = 10

In [ ]:
cit = kcit
results = ecit_simu_alpha(n_list=n_list, t=t, cit=cit, k=k, alphas=alphas)
show_results_choose(results, alphas)
results

In [ ]:
cit = cmiknn
results = ecit_simu_alpha(n_list=n_list, t=t, cit=cit, k=k, alphas=alphas)
show_results_choose(results, alphas)
results

In [ ]:
cit = ccit
results = ecit_simu_alpha(n_list=n_list, t=t, cit=cit, k=k, alphas=alphas)
show_results_choose(results, alphas)
results